In [ ]:
!python3 -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!python3 -m pip install tokenizers -U
!python3 -m pip install transformers -U

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import jsonlines

In [ ]:
# Load the tokenizer and model from Hugging Face

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  torch_dtype=torch.float32,
)

## looping over multiple prompts and logits

In [ ]:
# Save to disk
output_path = "golden_data_mixtral-8x7b-2.jsonl"


# Your prompt text
prompt_texts = ["[INST] I love to [/INST]", "[INST] Today is a [/INST]", "[INST] What is the [/INST]"]
all_data_to_save = []


for prompt_text in prompt_texts:
  # Encode the prompt text
  input_ids = tokenizer.encode(prompt_text, return_tensors="pt")

  # Get the logits for the prompt + completion
  with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits

    # Convert logits to fp32
    logits = logits.cpu().numpy().astype("float32")

    # Prepare data to be saved
    data_to_save = {
      "prompt": prompt_text,
      "tokens": input_ids.tolist()[0],
      "logits": logits.tolist()[0],  # Convert numpy array to list for JSON serialization
    }
    all_data_to_save.append(data_to_save)

with jsonlines.open(output_path, "w") as f:
  f.write_all(all_data_to_save)


print(f"Data saved to {output_path}")